In [1]:
from rateslib import *

# Definitions of dual numbers

In [4]:
z_x = Dual2(0.0, ["x"], [], [])
z_x

<Dual2: 0.000000, (x), [1.0], [[...]]>

In [5]:
z_x * z_x

<Dual2: 0.000000, (x), [0.0], [[...]]>

In [6]:
(z_x * z_x).dual2

array([[1.]])

# General functions of dual numbers

In [7]:
import math
def dual_sin(x: float | Dual) -> float | Dual:
    if isinstance(x, Dual):
        return Dual(math.sin(x.real), x.vars, math.cos(x.real) * x.dual)
    return math.sin(x)

In [8]:
x = Dual(2.1, ["y"], [])
dual_sin(x)

<Dual: 0.863209, (y), [-0.5]>

# Upcasting and dynamic variables

In [10]:
first_dual = Dual(11.0, ["x", "y"], [3, 8])
second_dual = Dual(-3.0, ["y", "z"], [-2, 5])
first_dual + second_dual + 2.65

<Dual: 10.650000, (x, y, z), [3.0, 6.0, 5.0]>

# First order derivatives and performance

In [20]:
def func(x, y, z):
    return x**6 + dual_exp(x/y) + dual_log(z)

x, y, z = 2.0, 1.0, 2.0
func(x, y, z)

72.0822032794906

In [21]:
%timeit func(x, y, z)

427 ns ± 7.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [22]:
x, y, z = Dual(2.0, ["x"], []), Dual(1.0, ["y"], []), Dual(2.0, ["z"], [])
func(x, y, z)

<Dual: 72.082203, (x, y, z), [199.4, -14.8, 0.5]>

In [26]:
%timeit func(x, y, z)

1.97 μs ± 78 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [27]:
x = Dual(2.0, ["x", "y", "z"], [1.0, 0.0, 0.0])
y = Dual(1.0, ["x", "y", "z"], [0.0, 1.0, 0.0])
z = Dual(2.0, ["x", "y", "z"], [0.0, 0.0, 1.0])

In [28]:
%timeit func(x, y, z)

1.91 μs ± 7.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [29]:
x = Dual(2.0, ["x", "y", "z"], [1.0, 0.0, 0.0])
y = Dual.vars_from(x, 1.0, ["x", "y", "z"], [0.0, 1.0, 0.0])
z = Dual.vars_from(x, 2.0, ["x", "y", "z"], [0.0, 0.0, 1.0])

In [30]:
%timeit func(x, y, z)

1.9 μs ± 6.84 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Numerical differentiation

In [31]:
def df_fwd_diff(f, x, y, z):
    base = f(x, y, z)
    dh = 1e-10
    dx = f(x+dh, y, z) - base
    dy = f(x, y+dh, z) - base
    dz = f(x, y, z+dh) - base
    return base, dx/dh, dy/dh, dz/dh

%timeit df_fwd_diff(func, 2.0, 1.0, 2.0)    

1.82 μs ± 2.89 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Functions with execution line delay

In [34]:
import time
def func_complex(x, y, z):
    time.sleep(0.000025)
    return x**6 + dual_exp(x/y) + dual_log(z)

%timeit func_complex(2.0, 1.0, 2.0)

33.3 μs ± 82.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [35]:
%timeit func_complex(x, y, z)

35 μs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [36]:
%timeit df_fwd_diff(func_complex, 2.0, 1.0, 2.0)

134 μs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Second order derivatives

In [38]:
x = Dual2(2.0, ["x", "y", "z"], [1.0, 0.0, 0.0], [])
y = Dual2(1.0, ["x", "y", "z"], [0.0, 1.0, 0.0], [])
z = Dual2(2.0, ["x", "y", "z"], [0.0, 0.0, 1.0], [])
func(x, y, z)

<Dual2: 72.082203, (x, y, z), [199.4, -14.8, 0.5], [[...]]>

In [39]:
gradient(func(x, y, z), ["x", "y"], order=2)

array([[487.3890561 , -22.1671683 ],
       [-22.1671683 ,  59.11244879]])

In [40]:
%timeit func(x, y, z)

3.99 μs ± 7.47 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
